# Evaluación Automatizada del Chatbot Turístico

Este notebook ejecuta automáticamente una serie de preguntas predefinidas a través del chatbot y guarda las respuestas para su análisis posterior.

In [27]:
# Importar bibliotecas necesarias
import os
import sys
import json
import pandas as pd
from dotenv import load_dotenv

# Configurar el path para encontrar los módulos
current_dir = os.path.dirname(os.path.abspath('__file__'))
src_dir = os.path.abspath(os.path.join(current_dir, '..'))
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)
print(f"Directorio src agregado al path: {src_dir}")

# Cargar variables de entorno
env_path = os.path.join(os.path.dirname(src_dir), '.env')
load_dotenv(env_path)
print(f"Archivo .env cargado desde: {env_path}")

Directorio src agregado al path: c:\Users\Janet\Documents\GitHub\GPTur\src
Archivo .env cargado desde: c:\Users\Janet\Documents\GitHub\GPTur\.env


In [28]:
# Verificar la configuración del entorno
def check_environment():
    checks = {
        "Python Path": src_dir in sys.path,
        "Módulo agents": False,
        "Módulo chatbot": False,
        "Módulo vector_db": False,
        ".env file": os.path.exists(env_path),
        "preguntas_evaluacion.csv": os.path.exists('preguntas_evaluacion.csv')
    }
    
    try:
        import agents
        checks["Módulo agents"] = True
    except ImportError as e:
        print(f"Error importando agents: {e}")
        
    try:
        import chatbot
        checks["Módulo chatbot"] = True
    except ImportError as e:
        print(f"Error importando chatbot: {e}")
        
    try:
        import vector_db
        checks["Módulo vector_db"] = True
    except ImportError as e:
        print(f"Error importando vector_db: {e}")
    
    print("\n=== Verificación del Entorno ===")
    all_passed = True
    for check, passed in checks.items():
        status = "✓" if passed else "✗"
        if not passed:
            all_passed = False
        print(f"{status} {check}")
    
    if not all_passed:
        print("\n⚠ Hay problemas en la configuración que deben ser resueltos")
    else:
        print("\n✓ Todo está correctamente configurado")
    
    return all_passed

# Ejecutar verificación
environment_ok = check_environment()


=== Verificación del Entorno ===
✓ Python Path
✓ Módulo agents
✓ Módulo chatbot
✓ Módulo vector_db
✗ .env file
✓ preguntas_evaluacion.csv

⚠ Hay problemas en la configuración que deben ser resueltos


In [29]:
# Importar el chatbot y sus componentes
from agents.agent_manager import AgentManager
from chatbot.core import CubaChatbot
from vector_db.chroma_storage import VectorStorage

# Inicializar el chatbot
def initialize_chatbot():
    try:
        print("\nInicializando Chatbot...")
        chatbot = CubaChatbot()
        print("Chatbot inicializado correctamente")
        
        return chatbot
    except ImportError as e:
        print(f"\nError de importación: {str(e)}")
        print("Verifica que:")
        print("1. Estás en el directorio correcto")
        print("2. El directorio 'src' está en el Python path")
        print(f"3. Los módulos están disponibles en: {src_dir}")
        return None
    except Exception as e:
        print(f"\nError inesperado: {str(e)}")
        print(f"Tipo de error: {type(e).__name__}")
        import traceback
        print("\nDetalles del error:")
        print(traceback.format_exc())
        return None

# Inicializar el chatbot
print("=== Iniciando configuración del chatbot ===")
chatbot = initialize_chatbot()

if chatbot is not None:
    print("\n✓ Chatbot inicializado exitosamente")
    print(f"Tipo de chatbot: {type(chatbot).__name__}")
else:
    print("\n✗ Error al inicializar el chatbot")

=== Iniciando configuración del chatbot ===

Inicializando Chatbot...


Inicializando Chatbot...
Chatbot inicializado correctamente

✓ Chatbot inicializado exitosamente
Tipo de chatbot: CubaChatbot
Chatbot inicializado correctamente

✓ Chatbot inicializado exitosamente
Tipo de chatbot: CubaChatbot


In [30]:
# Cargar las preguntas desde el archivo CSV
def load_questions():
    try:
        questions_df = pd.read_csv('preguntas_evaluacion.csv')
        return questions_df['pregunta'].tolist()
    except Exception as e:
        print(f"Error al cargar las preguntas: {str(e)}")
        return []

# Cargar las preguntas
questions = load_questions()
print(f"Se cargaron {len(questions)} preguntas")

Se cargaron 30 preguntas


In [31]:
# Función para procesar las preguntas y obtener respuestas
def process_questions(chatbot, questions):
    results = []
    
    for i, question in enumerate(questions, 1):
        try:
            print(f"\nProcesando pregunta {i}/{len(questions)}")
            print(f"Pregunta: {question}")
            
            # Obtener respuesta del chatbot
            print("Enviando pregunta al chatbot...")
            response = chatbot.generate_response(question)
            print("Respuesta recibida del chatbot")
            
            # Procesar la respuesta según su tipo
            if response is None:
                raise ValueError("La respuesta del chatbot es None")
                
            if isinstance(response, dict):
                if 'response' in response:
                    response_text = response['response']
                elif 'message' in response:
                    response_text = response['message']
                else:
                    print(f"Estructura de respuesta inesperada: {response.keys()}")
                    response_text = str(response)
            else:
                response_text = str(response)
            
            print(f"Longitud de la respuesta: {len(response_text)} caracteres")
            
            # Guardar resultado
            results.append({
                'pregunta': question,
                'respuesta_manual': response_text
            })
            
            # Guardar resultados parciales cada 5 preguntas
            if i % 5 == 0:
                save_partial_results(results, i)
                
        except Exception as e:
            print(f"Error procesando pregunta {i}: {str(e)}")
            results.append({
                'pregunta': question,
                'respuesta': f"ERROR: {str(e)}",
                'id': i
            })
    
    return results

# Función para guardar resultados parciales
def save_partial_results(results, current_question):
    partial_df = pd.DataFrame(results)
    
    # También actualizar el archivo principal para comparison.ipynb
    partial_df.to_csv('resultados_manuales.csv', index=False)
    print("Archivo principal 'resultados_manuales.csv' actualizado")

In [32]:
# Ejecutar el experimento
if chatbot is None:
    print("Error: El chatbot no se inicializó correctamente")
    print("Verifica que:")
    print("1. Todas las dependencias están instaladas")
    print("2. El path al directorio src es correcto")
    print("3. Las clases y módulos están correctamente importados")
elif not questions:
    print("Error: No se pudieron cargar las preguntas")
    print("Verifica que:")
    print("1. El archivo 'preguntas_evaluacion.csv' existe en el directorio actual")
    print("2. El archivo contiene una columna llamada 'pregunta'")
else:
    print(f"Chatbot inicializado: {type(chatbot).__name__}")
    print(f"Preguntas cargadas: {len(questions)}")
    print("\nIniciando procesamiento de preguntas...")
    
    results = process_questions(chatbot, questions)
    
    # Guardar resultados finales
    final_df = pd.DataFrame(results)
    final_df.to_csv('resultados_manuales.csv', index=False)
    print("\nProceso completado. Resultados guardados en 'resultados_manuales.csv'")
    
    # Mostrar algunas estadísticas
    print(f"\nEstadísticas:")
    print(f"Total de preguntas procesadas: {len(results)}")
    print(f"Preguntas con error: {len([r for r in results if 'ERROR' in str(r.get('respuesta_manual', ''))])}")
    
    # Verificar formato para comparison.ipynb
    if 'respuesta_manual' in final_df.columns and 'pregunta' in final_df.columns:
        print("\nFormato correcto para comparison.ipynb ✓")
    else:
        print("\nAdvertencia: El formato puede no ser compatible con comparison.ipynb")
        print("Columnas esperadas: 'pregunta', 'respuesta_manual'")
        print(f"Columnas actuales: {', '.join(final_df.columns)}")

Chatbot inicializado: CubaChatbot
Preguntas cargadas: 30

Iniciando procesamiento de preguntas...

Procesando pregunta 1/30
Pregunta: ¿Cómo puedo llegar a Cienfuegos desde Trinidad?
Enviando pregunta al chatbot...

Preguntas cargadas: 30

Iniciando procesamiento de preguntas...

Procesando pregunta 1/30
Pregunta: ¿Cómo puedo llegar a Cienfuegos desde Trinidad?
Enviando pregunta al chatbot...
Respuesta recibida del chatbot
Longitud de la respuesta: 1596 caracteres

Procesando pregunta 2/30
Pregunta: ¿Dónde puedo encontrar más información sobre el Nature Tour Nicho-Cienfuegos?
Enviando pregunta al chatbot...
Respuesta recibida del chatbot
Longitud de la respuesta: 1596 caracteres

Procesando pregunta 2/30
Pregunta: ¿Dónde puedo encontrar más información sobre el Nature Tour Nicho-Cienfuegos?
Enviando pregunta al chatbot...
Respuesta recibida del chatbot
Longitud de la respuesta: 609 caracteres

Procesando pregunta 3/30
Pregunta: ¿Qué lugares de interés hay en Matanzas?
Enviando pregunta 